# Named Entity Linking - premier test

In [2]:
# Faire importer les données. Je utilise seulement les articles titulés "Inland" pour avoir une
# plus hausse concentration des lieux domestiques. Cependant, il y reste beaucoup des lieux venants d'un peu partout

import pandas as pd

df = pd.read_excel('../data/processed/inland.xlsx')
df.rename(columns = {'Unnamed: 0':'id'}, inplace = True)
df.head()

,id,date,year,month,day,pub,head,full_text,href,text_len,w_count,sturm,hagel,überschwemmung,LOC
0,2247,1828.02.07,1828,2,7,Rigasche Zeitung,Inland.,"Inland.\n\nOdessa, den 14. Januar.\n\nDas Eis,...",http://periodika.lv/viewerOpener?issue=/p_003_...,3664,492,True,False,False,False
1,2283,1828.04.03,1828,4,3,Rigasche Zeitung,Inland.,"Inland.\n\nRiga, den 23. Marz.\n\nSeine Kaiser...",http://periodika.lv/viewerOpener?issue=/p_003_...,2314,328,False,False,False,False
2,5385,1845.01.03,1845,1,3,Zuschauer,Inland.,"Inland.\n\nSt. Petersburg, 30. December. Auf d...",http://periodika.lv/viewerOpener?issue=/dzus18...,25468,3188,True,False,False,False
3,5392,1845.01.10,1845,1,10,Zuschauer,Inland.,"Inland.\n\nRiga, 10. Januar. Einem osficiellen...",http://periodika.lv/viewerOpener?issue=/dzus18...,6306,748,True,False,False,False
4,5402,1845.01.23,1845,1,23,Zuschauer,Inland.,"Inland.\nInland.\n\nSt. Petersburg,- 17. Janua...",http://periodika.lv/viewerOpener?issue=/dzus18...,26761,3353,True,False,False,False


## 1

Dans un premier temps, je vais marquer les nom des lieux qui sont très répandus dans le dataset, mais qui ne sont pas dans la région concernée (pays baltes), pour éviter des faux-positifs. P. ex "Paris" est similaire à un village estonien appellé "Pari"...

In [11]:
def gather_locs(df):
    
    from tqdm import tqdm
    from climdist.ocr.spellcorrection import strip_word
    import time
    
    locations = []
    begin = time.time()
    
    for i in tqdm(range(0, len(df))):
        doc = nlp(df.iloc[i].full_text)
        
        for ent in doc.ents:
            if ent.label_ == 'LOC':
                locations.append(strip_word(ent.text, extra_symbols="?!|«»<>_'"))
                
    end = time.time()
    print(f'Execution time: {end-begin}')
    print(f'Average time per entry: {(end-begin)/len(df)}')
    
    return locations


def mark_loc_names(namelist):
    
    result = []
    ct = 0
    
    for name in namelist:
        ct += 1
        print('\n', ct, name)
        append = input('Abroad? [y]/n')
        if append != 'n':
            result.append(name)
            
        if ct%10 == 0:
            cont = input('----- Continue? [y]/n')
            if cont == 'n':
                break
                
    return result

In [ ]:
# prendre une échantillon et register les 300 noms le plus fréquents, puis écarter ceux "à l'étranger"

sample_data = df.sample(2000)
locs = gather_locs(sample_data)

#with open('../pipeline/03_ner/02_linker/locs_ignore.txt', 'w', encoding='utf8') as f:
#    f.write(str(locs))
    
sortedlocs = sorted(locs, key=locs.count, reverse=True)

uniques = []
for loc in sortedlocs:
    if loc not in uniques:
        uniques.append(loc)
        
#ignore_locs = mark_loc_names(uniques)

In [14]:
# Les lieux à ignorer selon la logique décrite sont:

with open('../pipeline/03_ner/02_linker/ignorelocs.txt', encoding='utf8') as f:
    ignore_locs = [line[:-1] for line in f.readlines()]
ignore_locs

['Paris',
 'Berlin',
 'Frankreich',
 'England',
 'London',
 'Petersburg',
 'Wien',
 'Deutschland',
 'Rußland',
 'St. Petersburg',
 'Rom',
 'Oesterreich',
 'Italien',
 'Moskau',
 'Preußen',
 'Europa',
 'Spanien',
 'Konstantinopel',
 'Warschau',
 'Odessa',
 'Hamburg',
 'China',
 'Frankreichs',
 'Polen',
 'Neapel',
 'Ungarn',
 'Amerika',
 'Rußlands',
 'Türkei',
 'Deutschlands',
 'Krim',
 'Belgien',
 'Peking',
 'Englands',
 'Leipzig',
 'Portugal',
 'Holland',
 'Madrid',
 'Pari',
 'Algier',
 'Schweiz',
 'München',
 'Amsterdam',
 'Donau',
 'Griechenland',
 'Wr',
 'Finnland',
 'Kiew',
 'Brüssel',
 'Dänemark',
 'Dresden',
 'Sewastopol',
 'Liverpool',
 'Wilna',
 'Königsberg',
 'Straßburg',
 'Europas',
 'Marseille',
 'Newyork',
 'Afrika',
 'Schweden',
 'Irland',
 'Stockholm',
 'Oesterreichs',
 'Kiel',
 'Washington',
 'Stettin',
 'Belgrad',
 'Athen',
 'Kopenhagen',
 'Lübeck',
 'Bulgarien',
 'Transvaal',
 'Lissabon',
 'Toulon',
 'Tuckum',
 'Serbien',
 'Köln',
 'Frankfurt',
 'Hannover',
 'Charkow',

## 2

Le linkage des lieux. J'utilise deux tabelles issues de Geonames.org qui contiennent environ 15 000 lieux pour Estonie et 10 000 pour Lettonie avec des cordonnées et, pour la plupart de cas, des noms alternatifs.

In [7]:
# Les fichiers et leur structure: les noms parallèls sont dans le 3e colonne, les X et Y dans les colonnes 4 et 5

ee_placenames = '../data/external/geonames/EE.tsv'
lv_placenames = '../data/external/geonames/LV.tsv'

ee = pd.read_csv(ee_placenames, sep='\t', encoding='utf8', header=None)
ee.sample(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
969,588385,Tammiku,Tammiku,Tammiku,58.93528,23.68500,P,PPL,EE,NaN,7.0,184.0,4063.0,NaN,0,NaN,11,Europe/Tallinn,2017-11-25
351,587767,Vaopere,Vaopere,Vaopere,59.06500,25.16056,P,PPL,EE,NaN,13.0,668.0,9050.0,NaN,0,NaN,84,Europe/Tallinn,2017-11-25
7278,8618093,Nassu,Nassu,NaN,58.26000,26.59000,L,LCTY,EE,NaN,18.0,528.0,8698.0,NaN,0,NaN,74,Europe/Tallinn,2017-11-25
699,588115,Udriku,Udriku,"Myza Udriku,Udriku,Udriku Asundus,Удрику",59.29694,26.07000,P,PPL,EE,EE,8.0,272.0,8622.0,NaN,0,NaN,96,Europe/Tallinn,2017-11-25
9743,8620559,Madaliku,Madaliku,"Madaliku,Wrangelshofscheirche",58.07917,26.81972,L,LCTY,EE,NaN,12.0,284.0,3505.0,NaN,0,NaN,119,Europe/Tallinn,2017-11-25


J'importe mon module de linkage. La logique de ce code est le suivante: 

1. Créer un document Spacy de chaque texte saisi
2. Trouver les entités nommées
3. (optionnel): créer des concordances autour des pairs des entités métérologiques, separés dans le texte par une distance "entdist" dans le texte. Si build_concordances = False, le script analyse le document entier, compte non tenu de la présence des entités de météo
4. Parcourir les noms et noms parallèles pour les comparer au texte de l'entité (avec un distance d'édition 1)

In [15]:
from climdist.ner.ner_linker import NERlinker
import spacy
nlp = spacy.load('../data/models/spacy_model_250421/')

In [36]:
linkage_data = df.sample(10)

linker = NERlinker(linkage_data,
                   nlp,
                   placename_files=['../data/external/geonames/EE.tsv', '../data/external/geonames/LV.tsv'],
                   ignorelocs=ignore_locs)

In [37]:
linker.link(entdist=40, window_before=60, window_after=60,
            output_path = '../pipeline/03_ner/02_linker/linking_demo_100521.json',
            stripwords=True,
            build_concordances=False,
            render=False)

------------------------------
Starting entry 25000 (1/10)
Düna Zeitung 1895.07.24
------------------------------
Applying NLP
LOC-s in concordance: {'Wasserklee', 'Antoinette', 'Ormancey', 'La Fontaine', 'Wi»d', 'kleiueu Insel'}
Searching for Wasserklee
No match found 

Searching for Antoinette
No match found 

Searching for Ormancey
No match found 

Searching for La Fontaine
No match found 

Searching for Wi»d
No match found 

Searching for kleiueu Insel
No match found 

{25000: {}} 


------------------------------
Starting entry 17539 (2/10)
Zeitung für Stadt und Land 1881.11.27
------------------------------
Applying NLP
LOC-s in concordance: {'Esset-n', 'Be-\nschiverdesachen', 'Wetdett;seln', 'Vuschwächtereien', 'Kurland', 'Essernsche Complex', 'Oesel', 'ein-er', 'Oesels', 'Wehftn.', 'Pampeln', 'Be-\nschwerdesechen', 'Odessa', 'Wehstn', 'OaltOhstnK rauss', 'Nigaschen Kreises', 'Arrende', 'St. u. Ld.«', 'Livland', 'Ackerland', 'Zahlbriese', 'Kronländereien', 'Werro', 'Riga', 'Bei-

No match found 

Passing Petersburg 

Passing Rußlands 

Passing London 

Searching for Galtfchika
No match found 

Passing Bulgarien 

Searching for Vardö
Best match: Vardi 

Passing Europa 

Searching for Cognac
No match found 

Passing Wr 

Searching for Riga
Best match: Riga 

Passing Deutschland 

{24099: {'Rusi': (455944, 57.10208, 23.00352), 'Insel Worms': (587455, 59.00528, 23.24639), 'Libau': (454432, 56.50806, 21.01149), 'Mitau': (459279, 56.65, 23.71278), 'Reval': (588406, 59.46663, 24.76702), 'Vardi': (587758, 59.03722, 24.45361), 'Riga': (456168, 56.92361, 23.97111)}} 


------------------------------
Starting entry 13737 (6/10)
Rigasche Zeitung 1871.06.26
------------------------------
Applying NLP
LOC-s in concordance: {'Deutschlands', 'Preußens', 'Europas', 'Rom', 'Rujen', 'Besitz', 'Memel', 'Frankreich', 'Herrenart', 'Griechenland', 'Berlins', 'Ausgaben', 'Libau', 'Wirthen', "Deutsch\n'", 'Gutsland', 'Preußen', 'Rußland', 'Berlin', 'Petersburg', 'Livland', 'Feldlazaret

No match found 

Passing Petersburg 

Searching for Jekateri-
nosslaw
No match found 

Searching for Estland
Best match: Estland 

Searching for Jekaterinofflaw
No match found 

Searching for Gute Wohlerlhof
No match found 

Searching for Riga
Best match: Riga 

{18259: {'Talsen': (454970, 57.24562, 22.58137), 'Frauenburg': (455890, 56.66363, 22.48807), 'Hasenpoth': (461602, 56.72108, 21.60156), 'Kurland': (460496, 57.0, 22.0), 'Friedrichstadt': (459402, 56.61319, 25.08316), 'Wolmar': (453754, 57.54108, 25.42751), 'Jakobstadt': (459282, 56.33333, 25.83333), 'Hapsal': (592224, 58.97944, 23.58278), 'Windau': (454308, 57.40188, 21.53778), 'Grobin': (459692, 56.53521, 21.16782), 'Libau': (454432, 56.50806, 21.01149), 'Sita': (455598, 57.17413, 27.10809), 'Mitau': (459279, 56.65, 23.71278), 'Reval': (588406, 59.46663, 24.76702), 'Ede': (460073, 56.87828, 22.06778), 'Dorpat': (588335, 58.38062, 26.72509), 'Estland': (453733, 59.0, 26.0), 'Riga': (456168, 56.92361, 23.97111)}} 




Regarder les données obtenues et les formatter pour pouvoir les saisir à QGIS

In [4]:
import json

with open('../pipeline/03_ner/02_linker/linking_demo_100521.json', 'r', encoding='utf8') as f:
    contents = f.read()
    data = json.loads(contents)
    
print(data)

{'25000': {}, '17539': {'Kurland': [460496, 57.0, 22.0], 'Oesel': [588878, 58.41333, 22.49639], 'OEsel': [588878, 58.41333, 22.49639], 'Livland': [453747, 57.16041, 24.26473], 'Werro': [587450, 57.83389, 27.01944], 'Riga': [456168, 56.92361, 23.97111]}, '30156': {'Kurland': [460496, 57.0, 22.0], 'Reval': [588406, 59.46663, 24.76702], 'Livland': [453747, 57.16041, 24.26473], 'Estland': [453733, 59.0, 26.0], 'Riga': [456168, 56.92361, 23.97111]}, '19489': {'Praga': [589350, 58.43389, 27.23583], 'Libau': [454432, 56.50806, 21.01149], 'Alt-Schwaneburg': [459668, 57.17767, 26.75291]}, '24099': {'Rusi': [455944, 57.10208, 23.00352], 'Insel Worms': [587455, 59.00528, 23.24639], 'Libau': [454432, 56.50806, 21.01149], 'Mitau': [459279, 56.65, 23.71278], 'Reval': [588406, 59.46663, 24.76702], 'Vardi': [587758, 59.03722, 24.45361], 'Riga': [456168, 56.92361, 23.97111]}, '13737': {'Rujen': [456008, 57.89752, 25.33155], 'Memele': [454425, 56.27686, 25.03215], 'Berzins': [461177, 56.96948, 21.63912]

In [6]:
def loc_data_to_tsv(data, output_path):
    
    with open(output_path, 'w', encoding='utf8') as f:
        
        f.write('name\tgeonames_id\ty\tx\n')
        
        for row in data:
            locations = data[row].items()
            #print(locations)
            
            for loc in locations:
                name = loc[0]
                #print(name)
                geonames_id = str(loc[1][0])
                y = str(loc[1][1])
                x = str(loc[1][2])
                
                line = (name + '\t' + geonames_id + '\t'+ y + '\t' + x + '\n')
                f.write(line)

In [7]:
loc_data_to_tsv(data, '../pipeline/03_ner/02_linker/linking_demo_100521.tsv')

In [10]:
# Faire un groupby pour compter des occurrences de chaque nom

locs_tsv = pd.read_csv('../pipeline/03_ner/02_linker/linking_demo_100521.tsv', sep='\t', encoding='utf8')
locs_tsv['count'] = 0
locs_tsv_counts = locs_tsv.groupby(['name', 'geonames_id', 'y', 'x']).count().reset_index()

locs_tsv_counts

,name,geonames_id,y,x,count
0,Alt-Schwaneburg,459668,57.17767,26.75291,1
1,Berzins,461177,56.96948,21.63912,1
2,Dorpat,588335,58.38062,26.72509,3
3,Ede,460073,56.87828,22.06778,1
4,Estland,453733,59.00000,26.00000,2
5,Frauenburg,455890,56.66363,22.48807,1
6,Friedrichstadt,459402,56.61319,25.08316,1
7,Goldingen,458460,56.97399,21.95721,1
8,Grobin,459692,56.53521,21.16782,1
9,Hapsal,592224,58.97944,23.58278,1


In [11]:
locs_tsv_counts.to_csv('../pipeline/03_ner/02_linker/linking_demo_100521_qgis.tsv', sep='\t', encoding='utf8')

## Analyse

Le linkage me semble d'être assez fiable pour un premier algorithme, mais pas encore appliquable. Comme le dataset est petit, je ne vais pas calculer la précision et rappel encore, mais je ferai quelques remarques sur les deux paramètres. Il y a des détails un peu compliquées de la géographie des pays baltes dans le paragraphes à suivre, mais ce n'est que pour illustrer des différents scénarios de ce problème complexe !

##### Précision

A première vue, 6-7 des noms me paraissent suspects (d'un total de 32 noms uniques). "Sita", "Stati" et "Ede" sont des noms assez improbables de paraître dans les journaux. "Saara" est un faux-positif, très proche à "Samara" en Russie. "Memel" est réconnu comme "Memele", un petit domaine en Lettonie, mais en fait "Memel" est le vieux nom allemand de "Klaipeda", la deuxième plus grande ville de *Lithouanie* aujourd'hui. La ville en question, appartenant à l'espace culturel baltique au 19e siècle, est bien présentée dans les journaux mais n'est pas reconnaissable pour le script, parce qu'elle ne se trouve pas dans les listes de *Lettonie* et *Estonie*.


##### Rappel

A part du rappel, il est possible de distinguer trois types des faux-négatifs :

- Les cas où le toponyme est accompagné d'un complément : p. ex., le script a bien réconnu "Libau", mais pas "Libauer Stadt" ("la ville de Libau"). Dans la même manière, "Aahof" est un mot composite qui signifie "le domaine d'Aa" (oui, il existe vraiment un village appelé "Aa" en Estonie!) et n'a pas été détecté.

- Particularités de grammaire : "Rigaschen Kreises" est le génitif de "Rigasche Kreis" (arrondissement de Riga, encore un exemple d'une division administrative obsolète). L'addition de n et s augmente la distance d'édition.

- Erreurs d'OCR. P. ex. "Soldtngen" devrait être "Goldingen", mais deux caractères sont erronéuses.


##### Remarques

La paramètre la plus importante avec un système de distance d'édition est la longueur du mot. Plus un mot a des caractères, plus la probabilité des erreurs d'OCR qui le rendent méconnaissable. Les noms plus longues sont aussi souvent des composites avec ses complications grammatiques. D'autre part, les noms courts ont des autres risques : une article ou préposition faussement identifé comme une entité risque d'être similaire à un toponyme.